IMPORT DES LIBRAIRIES NECESSAIRE

In [3]:
#installez la librairie 
#pip install bs4 requests

In [4]:
import os
import pandas as pd
import csv
import requests
from bs4 import BeautifulSoup
import numpy as np

In [5]:
#Obtient le contenu de la page html de l'url donnée
def getUrl(url):
  requete = requests.get(url)
  page = requete.content
  return page


#Récupération des tableaux de relations sortantes/entrantes
def getRelation(tab1):
  sortant = list()
  for i in tab1:
    #sortant.append(i.split(';')[3:6])
    sortant.append(i.split(';'))

  return sortant 

# Fonctions

In [6]:
import os.path
def presenceFichier(nomFichier) : 
    if os.path.isfile(nomFichier):
        return True 
    else:
        return False

In [7]:
#Cette fonction permet d'enregistrée le contenu de l'affichage de la page obtenu aprés recherche d'un mot dans un fichier texte 
def enregistrementFichier(a):
    
    #on vérifie que le mot n'est pas d'espace 
    a = a.replace(" ", "+")

    url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=mo&rel=?gotermsubmit=Chercher&gotermrel="+a+"&rel=3Fgotermsubmit%3DChercher"
    
    codeBuffer = BeautifulSoup(getUrl(url)).find("code")
    if (codeBuffer == None):
        return ''
    code = str(codeBuffer.text)
    
    
    if (presenceFichier(a+'.txt')):
        print("Fichier trouvé")
    else:
       # for i in range(len(a)-1):
       #     if a[i]=='>':
        
        a = a.replace('>',"")
        a = a.replace('<',"")
        print("Fichier non trouvé : on ajoute " + a)
        f = open(a+'.txt','w',encoding="utf8")
        b = str(code)
        f.write(b)
        
    file = open(a+'.txt',  encoding="utf8")
    text = file.readlines()
    return text
        
    
    

In [8]:
#Retourne le 5ème élément (qui est le poids) d'un tableau représentant une relation 
def takeWeight(elem):
  if(len(elem) > 5):
    return int(elem[5])

In [9]:
#Cette fonction permet de retourner un tableau contenant les relations sortantes du mot entrée en argument 

def getRelationSortanteb(mot):
    
    mot = mot.replace(" ", "+")
    #on verifie que le fichier existe bien 
    #sinon on le crée
    if(not presenceFichier(mot +'.txt')) :
        enregistrementFichier(mot)
        
    
    mot = mot.replace('>',"")
    mot = mot.replace('<',"")
    
    file = open(mot +'.txt', encoding="utf8")
    text = file.readlines()
    relationSortantebis = list()
    
    
    #On teste si il existe des relations sortantes 
    if(text.count('// les relations sortantes : r;rid;node1;node2;type;w \n')==0):
        return []
        
    else:
        
        sortantes = text.index('// les relations sortantes : r;rid;node1;node2;type;w \n')
        
        
    #On teste si il existe des relations entrente
    if(text.count('// les relations entrantes : r;rid;node1;node2;type;w \n')==0):
        entrantes = text.index('// END\n')
        
        
    else :
        entrantes = text.index('// les relations entrantes : r;rid;node1;node2;type;w \n')
    
   


    #si il n'y a pas de commentaire 
    
    relationSortante = text[sortantes+2 : entrantes-1]
    
    for i in relationSortante :
    
        relationSortantebis.append(i.replace('\n',""))

    relationSortantebis = getRelation(relationSortantebis)
    
    #si il y a des commentaires 
    for i in range(len(relationSortantebis)-1):
        if(len(relationSortantebis[i]) != 6):
            relationSortantebis.pop(i)
            
    relationSortantebis.sort(reverse = True, key = takeWeight)

      #on garde seulement les poids positifs
    sortantFinal = []
    for i in relationSortantebis : 
        if int(i[5]) > 0 : 
          sortantFinal.append(i)
        else : 
          break
    

    return sortantFinal

#Cette fonction permet de retourner un tableau contenant les relations entrantes du mot entrée en argument 

def getRelationEntranteb(mot):
    
    #on verifie que le fichier existe bien 
    #sinon on le crée 
    
    mot = mot.replace(" ", "+")

    if(not presenceFichier(mot +'.txt')) :
        enregistrementFichier(mot)
    
    mot = mot.replace('>',"")
    mot = mot.replace('<',"")
    
    file = open(mot +'.txt',  encoding="utf8")
    text = file.readlines()
    
   
    if(text.count('// les relations entrantes : r;rid;node1;node2;type;w \n')==0):
        return []
        
        
    else :
        
        entrantes = text.index('// les relations entrantes : r;rid;node1;node2;type;w \n')
    
    end = text.index('// END\n')

    relationEntrantebis = list()
    relationEntrante = text[entrantes +2 : end-1]
    for i in relationEntrante :
    
        relationEntrantebis.append(i.replace('\n',""))
        
    
    relationEntrantebis = getRelation(relationEntrantebis)
    for i in range(len(relationEntrantebis)-1):
        if(len(relationEntrantebis[i]) != 6):
            relationEntrantebis.pop(i)
    
    return relationEntrantebis



#Fonction qui récupère le nom d'une relation à partir de son id 
def get_id(a):

  url = "http://www.jeuxdemots.org/jdm-about-detail-relations.php"
  entite = BeautifulSoup(getUrl(url))
  name_r = list()
  for line in entite.find_all('td',{'info':'rel_name'}):
    name_r.append(line.text.replace('\t','').replace('\n','').strip())
  

  id_r =list()
  for col in entite.find_all('td',{'info':'rel_id'}):
    id_r.append(col.text)
  
  if a in name_r :
    
    return str(id_r[name_r.index(a)].split()[0])



In [10]:
#Renvoie l'id correspondant à un mot 
def correspondanceMotId(mot):
    text = []
    if(presenceFichier(mot+'.txt')):
        
        file = open(mot+'.txt',  encoding="utf8")
        text = file.readlines()
        
    else:
        text = enregistrementFichier(mot)
    
    
    debut = text.index("// les noeuds/termes (Entries) : e;eid;'name';type;w;'formated name' \n")
    fin = text.index("// les types de relations (Relation Types) : rt;rtid;'trname';'trgpname';'rthelp' \n")
    
    relationbis = list()
    relation = text[debut +2 : fin-1]
    for i in relation :
    
        relationbis.append(i.replace('\n',""))
        
    
    relationbis = getRelation(relationbis)
    
    
    for i in relationbis:
        
        if str(i[2]).replace('\'', '') == mot:
            return i[1]
    
    

   

correspondanceMotId('zebre')

#Renvoie le nom de l'id fournis 
def correspondanceIdMot(id,mot):
    if(presenceFichier(mot+'.txt')):
        
        file = open(mot+'.txt',  encoding="utf8")
        text = file.readlines()
        
    else:
         text = enregistrementFichier(mot)
        
    debut = text.index("// les noeuds/termes (Entries) : e;eid;'name';type;w;'formated name' \n")
    fin = text.index("// les types de relations (Relation Types) : rt;rtid;'trname';'trgpname';'rthelp' \n")
    
    relationbis = list()
    relation = text[debut +2 : fin-1]
    for i in relation :
    
        relationbis.append(i.replace('\n',""))
        
    
    relationbis = getRelation(relationbis)
    
    
    for i in relationbis:
        
        if int(i[1]) == int(id):
            
            return i[2].replace('\'', '')
        


# Transitivité

In [11]:
#1->2 2->3
#relativité à distance deux 
def transitive(mot1, relation, mot2):
  id = get_id(relation)
  id_mot = correspondanceMotId(mot2)
  chemin = []

  #On regarde les 15 premières relations sortantes
  iteration = 0

  #je suis dans les mots en relation avec mot1
  for i in getRelationSortanteb(mot1):
    #je restreins la relation à la relation entrée en argument 
    
    if (iteration < 1000):
        iteration = iteration + 1 

        if (str(i[4]) == id):
          print( mot1 +" "+ relation +" "+ correspondanceIdMot(i[3],mot1))
          mot = correspondanceIdMot(i[3],mot1)

          for i in getRelationSortanteb(mot) :
           

               if (str(i[3]) == id_mot):
                return "On a trouvé : "+mot1 +" " + relation + " "+ mot +" "+ relation+" "+ mot2+". Donc "+mot1+" "+relation+" "+mot2+" est vrai."

          #le mot n'a pas fonctionné donc on le place dans chemin 
          chemin.append(mot1)
    else : 
    #si ca fait plus de 15 relations qu'on test
      break
  return [chemin, [], None] 


# Déductivité

In [12]:
def deductif(mot1, mot2, relation) : 
  relation_id = get_id(relation)
  is_a_id = get_id('r_isa')
  var_id = get_id('r_variante')
  syn_id = get_id('r_syn')
  id_mot1 = correspondanceMotId(mot1)
  id_mot2 = correspondanceMotId(mot2)

  #On regarde les 15 premières relations sortantes et les 3 premières is_a
  iteration = 0  
  iteration_isa = 0

  #contiendra les génériques qui n'ont pas fonctionnés
  chemin = []

  #stocker le synonyme et les variantes du mot1 
  syn = None 
  var = []

  for i in getRelationSortanteb(mot1):

    if (iteration < 15):
      iteration = iteration + 1 

      #si on trouve une relation is_a 
      if (str(i[4]) == is_a_id):
        
        if (iteration_isa < 3):
          iteration_isa = iteration_isa + 1

          generique_mot = correspondanceIdMot(str(i[3]) ,mot1)

          #on regarde le generique de la relation is_a et on regarde si la relation generique relation mot2 fonctionne 
          for j in getRelationSortanteb(generique_mot):
            if (str(j[4]) == relation_id and str(j[3]) == id_mot2):           
              if chemin == [] : 
                return "On a trouvé : "+mot1+" r_isa "+generique_mot+" et "+generique_mot+" "+relation+" "+mot2+". Donc "+mot1+" "+relation+" "+mot2+" est vrai."
              else :
                chemin_total = ""
                for mot_chemin in chemin :
                  chemin_total= chemin_total+" "+mot_chemin+", "
                return "Après les génériques suivants : "+chemin_total+"on a trouvé : "+mot1+" r_isa "+generique_mot+" et "+generique_mot+" "+relation+" "+mot2+". Donc "+mot1+" "+relation+" "+mot2+" est vrai."

          #on enregistre le generique car il n'a pas fonctionné 
          chemin.append(generique_mot)
        else : 
          #si ca fait plus de 3 relations is_a
          break

      #si on trouve une relation r_variante 
      elif (str(i[4]) == var_id):
        var.append(correspondanceIdMot(str(i[3]),mot1))

      #si on trouve une relation r_syn
      elif(syn == None and str(i[4]) == syn_id):
        syn = correspondanceIdMot(str(i[3]),mot1)
    else : 
      #si ca fait plus de 15 relations qu'on test
      break
  
  return [chemin, var, syn]



# Inductivité

In [13]:
def inductif(mot1, mot2, relation) : 
  relation_id = get_id(relation)
  is_a_id = get_id('r_isa')
  hypo_id = get_id('r_hypo')
  var_id = get_id('r_variante')
  syn_id = get_id('r_syn')
  id_mot1 = correspondanceMotId(mot1)
  id_mot2 = correspondanceMotId(mot2)

  #On regarde les 15 premières relations sortantes et les 3 premières relations hypo/isa
  iteration = 0
  iteration_rel = 0
  
  #contiendra les spécifiques qui n'ont pas fonctionnés
  chemin = []

  #stocker le synonyme et les variantes du mot1
  syn = None 
  var = []

  for i in getRelationEntranteb(mot1):

    if (iteration < 15):
      iteration = iteration + 1 

      #si on trouve une relation is_a ou hypo
      if (str(i[4]) == hypo_id or str(i[4]) == is_a_id):     

        if (iteration_rel < 3):
          iteration_rel = iteration_rel + 1

            
          specifique_mot = correspondanceIdMot(str(i[2]),mot1)
          
          #on regarde le generique de la relation is_a et on regarde si la relation generique relation mot2 fonctionne 
          for j in getRelationSortanteb(specifique_mot):
            if (str(j[4]) == relation_id and str(j[3]) == id_mot2):
                
              if str(i[4]) == hypo_id :
                relation_trouve = "r_hypo"
              else : 
                relation_trouve = "r_isa"
                
              if chemin == [] : 
                return "On a trouvé : "+specifique_mot+" "+relation_trouve+" "+mot1+" et "+relation+" "+mot2+". Donc "+mot1+" "+relation+" "+mot2+" est vrai."
              else :
                chemin_total = ""
                for mot_chemin in chemin :
                  chemin_total= chemin_total+" "+mot_chemin+", "
                return "Après les spécifiques suivants : "+chemin_total+"on a trouvé : "+specifique_mot+" "+relation_trouve+" "+mot1+" et "+relation+" "+mot2+". Donc "+mot1+" "+relation+" "+mot2+" est vrai."

          #on enregistre le specifique car il n'a pas fonctionné
          chemin.append(specifique_mot)
        else :
          #si ca fait plus de 3 relations is_a ou hypo
          break

      #si on trouve une relation r_variante 
      elif (str(i[4]) == var_id):
        var.append(correspondanceIdMot(str(i[2]),mot1))

      #si on trouve une relation r_syn
      elif(syn == None and str(i[4]) == syn_id):
        syn = correspondanceIdMot(str(i[2]),mot1)

    else : 
      #si ca fait plus de 15 relations qu'on test
      break
  
  return [chemin, var, syn]


# Fonctions associées au menu 

In [14]:
#Vérification du choix que l'utilisateur a entré
def choixInferences(choix, terme1, terme2, relation) :
  if choix.isdigit() : 
    choix = int(choix)

  if choix == 1 : 
    return deductif(terme1, terme2, relation)
  elif choix == 2 : 
    return inductif(terme1, terme2, relation)
  elif choix == 3 : 
    return transitive(terme1,  relation,terme2)
  else :
    return None

#Arrangement de la relation 
def relation_arrangement(terme1, terme2, relation):
  r_a_changer = ['r_agent', 'r_patient', 'r_instr', 'r_own']
  r_a_changer2 = ['r_carac-1', 'r_lieu-1', 'r_manner-1', 'r_sentiment-1']

  if relation in r_a_changer : 
    return [terme2, terme1, relation+'-1']
  elif relation in r_a_changer2 : 
    return [terme2, terme1, relation[:relation.find('-1')]]
  else : 
    return None 

# Menu 

In [19]:

#On demande les arguments à l'utilisation et on vérifie s'ils existent'
relation_id = None 
terme1_id = '' 
terme2_id = ''

while relation_id == None or terme1_id == '' or terme2_id == '' :
  print("Veuillez entrer dans l'ordre un terme, une relation et enfin un terme !\n Exemple : pigeon r_agent-1 voler\n")
  a = input()

  a_total = a.split()
  terme1 = ""
  terme2 = ""
  relation = None 

  for element in a_total:
    if element.find("r_") == -1 and relation == None : 
      if terme1 == "":
        terme1 = element 
      else : 
        terme1 = terme1 + " " + element
    elif element.find("r_") == -1 :
      if terme2 == "":
        terme2 = element 
      else : 
        terme2 = terme2 + " "+ element 
    else :
      relation = element
    
  e1 = enregistrementFichier(terme1)
  e2 = enregistrementFichier(terme2)

  relation_id = get_id(relation)
  if e1 == '' or e2 == '':
        terme1 = e1
        terme2 = e2
  else :
      terme1_id = correspondanceMotId(terme1)
      terme2_id = correspondanceMotId(terme2)

  if relation_id == None : 
    print("La relation entrée n'existe pas.\n")
  elif terme1_id == '' : 
    print("Le premier terme ne se trouve pas sur jeux de mots. \n")
  elif terme2_id == '' :
    print("Le second terme ne se trouve pas sur jeux de mots.\n")
    
arrangement = relation_arrangement(terme1, terme2, relation)
if arrangement != None : 
  terme1 = arrangement[0]
  terme2 = arrangement[1]
  relation_origine = relation
  relation = arrangement[2]
    
#on enregistre sur un fichier si cela n'a pas deja été fait 


#Menu de choix proposés à l'utilisateur
choix = None 
result = None 
while choix == None :
  print("Tapez le chiffre du type de l'inférence que vous voulez tester : \n1 : Déductive\n2 : Inductive\n3 : Transitive\n")
  choix = input()
  result = choixInferences(choix, terme1, terme2, relation)
  if result == None : 
    choix = None 

#si result est un tableau alors il n'a pas trouvé d'inférence justifiant la relation  
if type(result) == type([]):

  #Affichage du résultat obtenu
  print("Nous n'avons pas trouvé une inférence qui affirmait la relation proposée, donc nous pensons qu'elle est fausse.")

  #dans le cas où il a étudié des mot 
  if result[0] != []:
    print("Nous n'avons pas trouvé que la relation fonctionnait pour les mots suivants : ")
    for i in result[0]: 
      print("- "+i)
  #le nombre d'itération a atteint sa limite
  else :
    print("Aucun des mots les plus proches au mot choisi n'avait la relation correcte.")

  #On peut voir si on arrive à trouver une solution avec les variantes puis le synonyme 
  if result[1] != [] or result[2] != None:

    print("Voulez-vous une recherche approfondie (O pour oui) ?")
    lettre_choix = input()

    if lettre_choix == 'O' or lettre_choix == 'o' : 
      #on fait une recherche avec les variantes et synonymes obtenues et stockés dans result 
      #c'est seulement possible pour le terme1 car on a stockés ses données en même temps que la première recherche de relation 
      #or il n'y a pas de recherche de relation pour le terme2 

      #on commence par les variantes 
      result2 = None
      motResult = None 

      for i in result[1] : 
          result2 = choixInferences(choix, i, terme2, relation)
          
          if type(result2) != type([]):
            motResult = i
            break

      #s'il n' y avait pas de variante ou que les variantes n'ont pas abouti à un vrai ET qu'il existe un synonyme 
      if (result2 == None or type(result2) == type([])) and result[2] != None : 
          result2 = choixInferences(choix, result[2], terme2, relation)

      #Affichage du résultat obtenu
      if type(result2) != type([]):
        #Affichage du mot variant ou synonyme 
        if motResult == None : 
          print("L'inférence a fonctionné pour le synonyme : "+result[2]+" du mot "+terme1)
        else : 
          print("L'inférence a fonctionné pour la variante : "+motResult+" du mot "+terme1)

        print(result2)
        if arrangement != None : 
          print('On peut donc conclure que : '+terme2+" "+relation_origine+" "+terme1+" est vraie.")  

      else : 
        #Affichage du résultat obtenu
        print("Nous n'avons toujours pas trouvé une inférence qui affirmait la relation proposée, donc nous la considérons fausse.")

        if( result[1] != []):
          print("Nous avons pourtant essayé avec les variantes du mot suivantes : \n")
          for i in result[1]: 
            print("- "+i)
        elif (result[2] != None):
          print("Nous avons essayé avec le synonyme du mot : "+result[2]+"\n")

else : 
  print(result)
  if arrangement != None : 
    print('On peut donc conclure que : '+terme2+" "+relation_origine+" "+terme1+" est vraie.")  
    
    
print("\nA bientôt !\n")

Veuillez entrer dans l'ordre un terme, une relation et enfin un terme !
 Exemple : pigeon r_agent-1 voler

Tour Eiffel r_lieu France
Fichier non trouvé : on ajoute Tour+Eiffel
Fichier non trouvé : on ajoute France
Fichier trouvé
Tapez le chiffre du type de l'inférence que vous voulez tester : 
1 : Déductive
2 : Inductive
3 : Transitive

3
Fichier trouvé
Tour Eiffel r_lieu Paris>159560
Fichier trouvé
Fichier non trouvé : on ajoute Paris159560
On a trouvé : Tour Eiffel r_lieu Paris>159560 r_lieu France. Donc Tour Eiffel r_lieu France est vrai.

A bientôt !

